# Image Caption Generator (InceptionV3 and COCO-2017 dataset)

## Importing Libraries

In [2]:
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
import glob
from PIL import Image
from time import time
import collections
import random
import numpy as np
import json
import tensorflow as tf

from tensorflow.keras import Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Dense, Activation, Flatten, Reshape, Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import add
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
#from tensorflow.keras.applications.vgg19 import preprocess_input
# from tensorflow.keras.applications.inception_v3 import preprocess_input
# from tensorflow.keras.applications. import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import keras
from tqdm import tqdm

## Importing Dataset and data preprocessing

In [9]:
train_images = '../input/coco-2017-dataset/coco2017/train2017/'
test_images = '../input/coco-2017-dataset/coco2017/test2017/'
val_images = '../input/coco-2017-dataset/coco2017/val2017/'

glove_path = '../input/glove6b/glove.6B.200d.txt'

In [10]:
train_images_len = len(os.listdir(train_images))
test_images_len = len(os.listdir(test_images))
val_images_len = len(os.listdir(val_images))

print(train_images_len)
print(test_images_len)
print(val_images_len)

In [11]:
annotation_file = '../input/coco-2017-dataset/coco2017/annotations/captions_train2017.json'
with open(annotation_file, 'r') as f:
    annotations = json.load(f)
print(annotations['annotations'][0])

In [12]:
annotation_file_val = '../input/coco-2017-dataset/coco2017/annotations/captions_val2017.json'
with open(annotation_file_val, 'r') as f:
    annotations_val = json.load(f)
print(annotations_val['annotations'][0])

In [13]:
# Group all captions together having the same image ID.
image_path_to_caption = collections.defaultdict(list)
for val in annotations['annotations']:
  caption = (f"{val['caption']}")
  image_path = train_images + '%012d.jpg' % (val['image_id'])  
  image_path_to_caption[image_path].append(caption)

In [15]:
# Group all captions together having the same image ID.
image_path_to_caption_val = collections.defaultdict(list)
for val in annotations_val['annotations']:
  caption = (f"{val['caption']}")
  image_path = val_images + '%012d.jpg' % (val['image_id'])  
  image_path_to_caption_val[image_path].append(caption)

In [16]:
print(len(image_path_to_caption))
image_path_to_caption['../input/coco-2017-dataset/coco2017/train2017/000000519186.jpg']

In [17]:
print(len(image_path_to_caption_val))
image_path_to_caption_val['../input/coco-2017-dataset/coco2017/val2017/000000000139.jpg']

In [18]:
# conversion from defaultdict to dict
image_path_to_caption = dict(image_path_to_caption)
print(type(image_path_to_caption))

In [19]:
# conversion from defaultdict to dict
image_path_to_caption_val = dict(image_path_to_caption_val)
print(type(image_path_to_caption_val))

## Dropping of some data from the traning set

In [20]:
import random

lst = list(image_path_to_caption.items())
#Run only one time ---
random.shuffle(lst)
x = int(len(lst)*0.5)
lst  = lst[0:x]
# ---------
image_path_to_caption = dict(lst)
print(len(image_path_to_caption))

In [21]:
import random

lst = list(image_path_to_caption_val.items())
#Run only one time ---
random.shuffle(lst)
x = int(len(lst)*0.5)
lst  = lst[0:x]
# ---------
image_path_to_caption_val = dict(lst)
print(len(image_path_to_caption_val))

### Image_id_to_caption Function so that you can convert path_to_caption to id_to_caption whenever you want to

In [33]:
def id_caption(image_path_to_caption): 
    image_id_to_caption = collections.defaultdict(list)
    for (key,val) in image_path_to_caption.items(): 
        for values in val:
            x = key.split('/')[-1]
            x = x.split('.')[0]
            image_id_to_caption[x].append(values)

    # Convert image_id_to_caption to dict
    image_id_to_caption = dict(image_id_to_caption)
    return image_id_to_caption

In [22]:
# Clean description means remove puncutation and lowercase

#1. image_path_to_caption
table = str.maketrans('', '', string.punctuation)
for key, desc_list in image_path_to_caption.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [w.translate(table) for w in desc]
        desc_list[i] =  ' '.join(desc)

In [32]:
# Clean description means remove puncutation and lowercase

#1. image_path_to_caption
table = str.maketrans('', '', string.punctuation)
for key, desc_list in image_path_to_caption_val.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [w.translate(table) for w in desc]
        desc_list[i] =  ' '.join(desc)

In [25]:
items = list(image_path_to_caption.items())
print(items[7])

In [26]:
items = list(image_path_to_caption_val.items())
print(items[7])

In [27]:
# unique words in our caption data
vocabulary = set()
for key in image_path_to_caption.keys():
        [vocabulary.update(d.split()) for d in image_path_to_caption[key]]
print(len(vocabulary))


In [28]:
# unique words in our caption data
vocabulary = set()
for key in image_path_to_caption_val.keys():
        [vocabulary.update(d.split()) for d in image_path_to_caption_val[key]]
print(len(vocabulary))


In [34]:
# Now call the id_caption function
image_id_to_caption = id_caption(image_path_to_caption)
print(len(image_id_to_caption))
print(type(image_id_to_caption))
print(list(image_id_to_caption.keys())[:5])

In [36]:
# Now call the id_caption function
image_id_to_caption_val = id_caption(image_path_to_caption_val)
print(len(image_id_to_caption_val))
print(type(image_id_to_caption_val))
print(list(image_id_to_caption_val.keys())[:5])

In [38]:
lines = list()
for key, desc_list in image_id_to_caption.items():
    for desc in desc_list:
        lines.append(key + ' ' + desc)
new_descriptions = '\n'.join(lines)

print(type(new_descriptions))
print(new_descriptions[:400])

In [39]:
lines = list()
for key, desc_list in image_id_to_caption_val.items():
    for desc in desc_list:
        lines.append(key + ' ' + desc)
new_descriptions_val = '\n'.join(lines)

print(type(new_descriptions_val))
print(new_descriptions_val[:400])

In [40]:
## Getting image ids
train = list(image_id_to_caption.keys())
print(train[0:5])

In [41]:
train_img = list(image_path_to_caption.keys())
print(len(train_img))

In [42]:
## Getting image ids
val = list(image_id_to_caption_val.keys())
print(val[0:5])

In [43]:
val_img = list(image_path_to_caption_val.keys())
print(len(val_img))

In [44]:
train_descriptions = dict()
for line in tqdm(new_descriptions.split('\n')):
    tokens = line.split()
    image_id, image_desc = tokens[0], tokens[1:]
    if image_id in train:
        if image_id not in train_descriptions:
            train_descriptions[image_id] = list()
        desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
        train_descriptions[image_id].append(desc)

print(len(train_descriptions))

In [46]:
val_descriptions = dict()
for line in tqdm(new_descriptions_val.split('\n')):
    tokens = line.split()
    image_id, image_desc = tokens[0], tokens[1:]
    if image_id in val:
        if image_id not in val_descriptions:
            val_descriptions[image_id] = list()
        desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
        val_descriptions[image_id].append(desc)

print(len(val_descriptions))

In [47]:
list(train_descriptions.keys())[0:5]

In [49]:
list(val_descriptions.keys())[0:5]

In [82]:
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)

In [83]:
all_val_captions = []
for key, val in val_descriptions.items():
    for cap in val:
        all_val_captions.append(cap)

In [84]:
print(len(all_train_captions)) # contain all the captions   5*11858 = 59173
print(all_train_captions[:5])

In [85]:
print(len(all_val_captions)) # contain all the captions   5*11858 = 59173
print(all_val_captions[:5])

In [86]:
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('Vocabulary = %d' % (len(vocab)))

In [87]:
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_val_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
vocab_val = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('Vocabulary = %d' % (len(vocab_val)))

In [133]:
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1

In [134]:
vocab_size

In [137]:
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab_val:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size_val = len(ixtoword) + 1

In [138]:
vocab_size_val 

In [90]:
all_desc = list()
for key in train_descriptions.keys():
    [all_desc.append(d) for d in train_descriptions[key]]
lines = all_desc
max_length = max(len(d.split()) for d in lines)

print('Description Length: %d' % max_length)

In [91]:
all_desc_val = list()
for key in val_descriptions.keys():
    [all_desc_val.append(d) for d in val_descriptions[key]]
lines = all_desc
max_length_val = max(len(d.split()) for d in lines)

print('Description Length: %d' % max_length_val)

In [92]:
embeddings_index = {} 
f = open(glove_path, encoding="utf-8")
for line in f:   
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

In [95]:
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [96]:
embedding_matrix[5].shape

## Model (InceptionV3)

In [97]:
# model = InceptionV3(weights='imagenet')
# model = VGG19(include_top = False, weights = 'imagenet')
model = ResNet50V2(weights='imagenet')
model_new = Model(model.input, model.layers[-2].output)

In [98]:
monitor_val_acc = EarlyStopping(monitor = 'val_loss', patience = 8)
checkpoint_filepath='image_caption_generator.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    save_best_only=True)

In [99]:
from keras.preprocessing.image import load_img,img_to_array
def preprocess(image_path):
    img = load_img(image_path, target_size=(299, 299))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [100]:
# train img is array of path value to images
print(len(train_img))
print(train_img[0:5])

In [101]:
def encode(image):
    image = preprocess(image) 
    fea_vec = model_new.predict(image) 
    #print(fea_vec.shape)
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    #print(fea_vec.shape)
    return fea_vec

In [114]:
encoding_train = {}
for img in tqdm(train_img):
    path = img.split('/')[-1]
    encoding_train[path] = encode(img)
train_features = encoding_train

# encoding_test = {}
# for img in test_img:
#     encoding_test[img[len(test_images):]] = encode(img)

In [115]:
encoding_val = {}
for img in tqdm(val_img):
    path = img.split('/')[-1]
    encoding_val[path] = encode(img)
val_features = encoding_val

In [108]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
      
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()

In [116]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [110]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [187]:
vocab_size_val=5448

In [143]:
def data_generator2(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size_val)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [188]:
epochs = 2
batch_size = 32
steps = len(train_descriptions)//batch_size
steps_val = len(val_descriptions)//batch_size

generator = data_generator(train_descriptions, train_features, wordtoix, max_length, batch_size)
generator_val = data_generator2(val_descriptions, val_features, wordtoix_val, max_length_val, batch_size)

history=model.fit(x=generator, epochs=epochs, steps_per_epoch=10,callbacks=[model_checkpoint_callback], verbose=1,validation_data=generator_val,validation_steps=steps_val)

In [189]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [193]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [194]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [167]:
print(history.history.keys())


In [190]:
plt.plot(history.history['accuracy'][0])
plt.plot(history.history['accuracy'][1])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [191]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [192]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break

    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
def beam_search_predictions(image, beam_index = 3):
    start = [wordtoix["startseq"]]
    start_word = [[start, 0.0]]
    while len(start_word[0][0]) < max_length:
        temp = []
        for s in start_word:
            par_caps = sequence.pad_sequences([s[0]], maxlen=max_length, padding='post')
            preds = model.predict([image,par_caps], verbose=0)
            word_preds = np.argsort(preds[0])[-beam_index:]
            # Getting the top <beam_index>(n) predictions and creating a 
            # new list so as to put them via the model again
            for w in word_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
                prob += preds[0][w]
                temp.append([next_cap, prob])
                    
        start_word = temp
        # Sorting according to the probabilities
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        # Getting the top words
        start_word = start_word[-beam_index:]
    
    start_word = start_word[-1][0]
    intermediate_caption = [ixtoword[i] for i in start_word]
    final_caption = []
    
    for i in intermediate_caption:
        if i != 'endseq':
            final_caption.append(i)
        else:
            break

    final_caption = ' '.join(final_caption[1:])
    return final_caption

## Saving the model

In [ ]:
# Saving Model to disk
model.save("image_caption_generator.h5")
print("Saved model to disk")

## Testing of the Model

In [ ]:
# This function will generate caption for the image 
def generate_caption(image_path):
    image = e(image_path)
    image = image.reshape((1, 2048))
    x=plt.imread(image_path)
    plt.imshow(x)
    plt.show()
    print("Greedy:",greedySearch(image))
    print("Beam Search, K = 3:",beam_search_predictions(image, beam_index = 3))
    print("Beam Search, K = 5:",beam_search_predictions(image, beam_index = 5))
    print("Beam Search, K = 7:",beam_search_predictions(image, beam_index = 7))
    

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/train2017/000000000009.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/train2017/000000000030.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/val2017/000000000776.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/val2017/000000000885.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/test2017/000000000063.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/test2017/000000000155.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/val2017/000000000785.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/test2017/000000000178.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/test2017/000000000202.jpg')

In [ ]:
generate_caption('../input/coco-2017-dataset/coco2017/test2017/000000000345.jpg')

In [ ]:
model.evaluate(generator,callbacks=[monitor_val_acc,model_checkpoint_callback], verbose=1,)


In [ ]:
model.evaluate(generator_val,callbacks=[monitor_val_acc,model_checkpoint_callback], verbose=1,)
